# 🚀 How to automate Excel using Python

> In this video, I will be sharing my favorite ways to **automate Microsoft Excel using Python**. In particular, we will be using the open-source tools: `Pandas`, `xlwings` & `plotly`. After this video, you will be able to create a custom Python script that allows you to combine excel files & create charts out of them.

***

# 0. Install Dependencies

In [ ]:
#!pip install pandas --quiet
#!pip install openpyxl --quiet
#!pip install plotly-express --quiet
#!pip install xlwings --quiet

# 1. Merge Excel Files (Simple)

### 📢 Purpose: <br><br> Merge multiple Excel files, perform calculations & export an interactive chart

In [41]:
from pathlib import Path
import pandas as pd  # pip install pandas
import plotly.express as px  # pip install plotly-express

In [42]:
# Locate examples files
INPUT_DIR = Path.cwd() / "1-Merge-Excel-Files-Simple" / "INPUT"
OUTPUT_DIR = Path.cwd() / "1-Merge-Excel-Files-Simple"

In [44]:
# Create empty dataframe, iterate over input directory, append all excel files to empty dataframe
df = pd.DataFrame()

for file in INPUT_DIR.iterdir():
    df = df.append(pd.read_excel(file), ignore_index=True)
    
df['Country'].unique()

array(['Canada', 'France', 'Germany', 'Japan', 'United States of America'],
      dtype=object)

In [45]:
df.head()

,Segment,Country,Product,Discount Band,Units Sold,Manufacturing Price,Sale Price,Gross Sales,Discounts,Sales,COGS,Profit,Date,Month Number,Month Name,Year
0,Government,Canada,Carretera,None,1618.5,3,20,32370.0,0.0,32370.0,16185.0,16185.0,2021-01-01,1,January,2021
1,Channel Partners,Canada,Montana,None,2518.0,5,12,30216.0,0.0,30216.0,7554.0,22662.0,2021-06-01,6,June,2021
2,Enterprise,Canada,Montana,None,2665.5,5,125,333187.5,0.0,333187.5,319860.0,13327.5,2021-07-01,7,July,2021
3,Government,Canada,Paseo,None,292.0,10,20,5840.0,0.0,5840.0,2920.0,2920.0,2021-02-01,2,February,2021
4,Channel Partners,Canada,Paseo,None,2518.0,10,12,30216.0,0.0,30216.0,7554.0,22662.0,2021-06-01,6,June,2021


In [46]:
df = df.groupby(by="Country").sum()[[" Sales", "Profit"]]
df

,Sales,Profit
Country,,
Canada,1.971177e+07,2725557.105
France,1.922138e+07,2969688.610
Germany,1.727761e+07,2562169.350
Japan,1.619493e+07,2314852.850
United States of America,1.990542e+07,2442969.835


In [47]:
df["Profit Margin %"] = (df["Profit"] / df[" Sales"]) * 100
df

,Sales,Profit,Profit Margin %
Country,,,
Canada,1.971177e+07,2725557.105,13.827057
France,1.922138e+07,2969688.610,15.449926
Germany,1.727761e+07,2562169.350,14.829424
Japan,1.619493e+07,2314852.850,14.293688
United States of America,1.990542e+07,2442969.835,12.272891


In [48]:
# Plot Profit situation & export grouped dataframe to Excel
fig = px.bar(
    df,
    x=df.index,
    y=" Sales",
    color="Profit Margin %",
    color_continuous_scale=["red", "yellow", "green"],
    template="plotly_white",
    title="<b>Profit Analysis</b>",
)
fig.write_html(str(OUTPUT_DIR / "profit_analysis.html"))
df.to_excel(OUTPUT_DIR / "summary.xlsx")
fig

### 🚨 Some Remarks

- Making use of the different plotting libraries, e.g. interactive visualizations using Plotly
- **Simplified Example** | In real-life, more often than not, datasets are not so well-structured

# 2. Merge Excel Files (Real Life Example)

### 📢 Purpose: <br><br>Merge multiple Excel files, perform calculations & insert (Excel) charts [keep excel formatting & formulas]

In [49]:
from pathlib import Path
import pandas as pd  # pip install pandas
import xlwings as xw  # pip install xlwings

In [50]:
# Locate examples files
INPUT_DIR = Path.cwd() / "2-Merge-Excel-Files-Real-Life" / "INPUT"
OUTPUT_DIR = Path.cwd() / "2-Merge-Excel-Files-Real-Life"

In [51]:
# Create an empty workbook. This wb will be our final output
summary_wb = xw.Book()
summary_sht = summary_wb.sheets[0]
summary_sht.name = "Summary"

In [52]:
# Iterate over each file in directory. Copy cell range & paste it into summary workbook (keep formatting & formuals)
for file in INPUT_DIR.iterdir():
    wb = xw.Book(file)
    wb.app.display_alerts = False
    #wb.app.visible = False
    sht = wb.sheets("Overview")
    values = sht.range("B5").expand().copy()
    dest_last_row = summary_sht.range("A1").expand().last_cell.row + 1
    summary_sht.range(f"A{dest_last_row}").paste()
    wb.close()

In [53]:
# Insert & style header of our summary workbook
header = [
    "Segment",
    "Country",
    "Product",
    "Discount Band",
    "Units Sold",
    "Sale Price",
    "Gross Sales",
    "Discounts",
    "Sales",
    "COGS",
    "Profit",
]
header_row = summary_sht.range("A1:K1")
header_row.value = header
header_row.font.bold = True
header_row.font.color = (255, 255, 255)
header_row.color = (119, 136, 153)

In [54]:
# Use pandas to perform calculations, e.g. data aggregation
df = summary_sht.range("A1").options(pd.DataFrame, expand="table").value
data_by_country = df.groupby(by="Country").sum()[["Sales"]]
data_by_country

,Sales
Country,
Canada,1.971177e+07
France,1.922138e+07
Germany,1.727761e+07
Japan,1.619493e+07
United States of America,1.990542e+07


In [55]:
# Insert grouped dataframe into our summary workbook
summary_sht.range("M1").value = data_by_country

In [56]:
# Insert EXCEL chart
chart = summary_sht.charts.add(
    left=summary_sht.range("M8").left, 
    top=summary_sht.range("M8").top,
    width=400,
    height=200,
)
chart.set_source_data(summary_sht.range("M1").expand())
chart.chart_type = "bar_stacked"

In [57]:
# Insert PANDAS chart
ax = data_by_country.sort_values(by="Sales").plot(kind="barh")
fig = ax.get_figure()
summary_sht.pictures.add(
    fig,
    name="Sales",
    update=True,
    left=summary_sht.range("M23").left,
    top=summary_sht.range("M23").top,
    width=400,
    height=200,
)

<Picture 'Sales' in <Sheet [Book1]Summary>>

In [58]:
# Clean up workbook, save workbook & quit Excel instance (if it is the only wb open)
summary_sht.autofit()
summary_wb.save(OUTPUT_DIR / "summary.xlsx")
if len(summary_wb.app.books) == 1:
    summary_wb.app.quit()
else:
    summary_wb.close()

### 🚨 Some Remarks

 **`xlwings` ..**
 - .. is just a smart wrapper around `pywin32` on Windows and `appscript` on Mac.
 - .. works nicely together with Pandas
 - .. (Excel) is not ideal when dealing with larger datasets.
 - .. has actually much more to offer. You can, for instance:
     - write user-defined-functions (UDF) in Python
     - run Python directly in Excel

# 3. My take on Excel automation in general & VBA vs. Python ⚡

## Should you try to automate all Excel files?

![meme](0-Images/meme-automation1.jpg "Meme")

---

### 🚨 Use the right tool for the right job!

#### 👉 MS Excel, PowerQuery, PowerBI, Database, VBA, Python, ...

![tool](0-Images/right-tool.jpeg "Right Tool")

## VBA or Python? Which one should I use?

**Overall:** <br>
> Computer programming languages are like **tools in a toolbox**. Different tools are designed to solve different problems.
- Python is a General-Purpose Language
- Visual Basic for Applications (VBA) is a simple, but powerful programming language to extend Office applications

**VBA:**<br>
  👍 Syntax is very intuitive. Example: Range("A1:C7").ClearContents <br>
  👍 No additional installation is required (huge advantage!). Built in `Visual Basic Editor (VBE)` <br>
  👎 It is restricted to Office Applications <br>

**Python:**<br>
  👍 Has very powerful packages (for data analysis, machine-/deeplearning, web development, ..) <br>
  👍 Python is one of the fastest-growing programming languages <br>
  👎 Requires additional setup: Python(distribution), Editor (other than Word or Notepad 😅)<br>
 
___
Python might be more powerful than VBA but like I said earlier, what matters is what you intend to achieve. There is no need killing a mosquito with an atomic bomb when a simple spray can do the job.

At times I find VBA-Excel more appropriate for a certain task and much more efficient than Python. But I also like the powerful Python packages.

# 4. Connect with me 💬

> 📺 **YouTube:** https://youtube.com/c/codingisfun <br>
> 🌎 **Website:** https://pythonandvba.com <br>
> 📝 **GitHub:** https://github.com/Sven-Bo <br>
> ⭐ **Discord:** https://pythonandvba.com/discord <br>
> 💬 **EMail:** contact@pythonandvba.com <br>